In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, date

# from sklearn.model_selection import train_test_split

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_absolute_error
# from sklearn.model_selection import GridSearchCV
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression


In [5]:
# Read the data
train = pd.read_csv('./sales_train.csv')
test = pd.read_csv('./test.csv')
submission = pd.read_csv('./sample_submission.csv')
items = pd.read_csv('./items.csv')
item_cats = pd.read_csv('./item_categories.csv')
shops = pd.read_csv('./shops.csv')

In [6]:
train = train.set_index('item_id').join(items.set_index('item_id')).drop('item_name', axis=1).reset_index()
train

,item_id,date,date_block_num,shop_id,item_price,item_cnt_day,item_category_id
0,0,01.09.2014,20,54,58.0,1.0,40
1,1,04.04.2014,15,55,4490.0,1.0,76
2,1,02.04.2014,15,55,4490.0,1.0,76
3,1,06.07.2014,18,55,4490.0,1.0,76
4,1,04.08.2014,19,55,4490.0,1.0,76
...,...,...,...,...,...,...,...
2935844,22168,02.02.2013,1,42,799.0,1.0,62
2935845,22168,17.02.2013,1,16,799.0,1.0,62
2935846,22168,08.03.2013,2,43,798.5,1.0,62
2935847,22168,13.09.2013,8,12,799.0,1.0,62


In [7]:
testFrame = pd.DataFrame(columns=['day','month','year'])

In [5]:
testFrame['day']= np.array(train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%d')), dtype=np.int8)

KeyboardInterrupt: 

In [10]:
testFrame

,day,month,year
0,9,1,14
1,4,4,14
2,4,2,14
3,7,6,14
4,8,4,14
...,...,...,...
2935844,2,2,13
2935845,2,17,13
2935846,3,8,13
2935847,9,13,13


In [21]:

start = time.perf_counter()

train['day']= np.array(train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%d')), dtype=np.int8)
train['month']= train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%m'))
train['year']= train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%Y'))
finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

Finished in 107.52 second(s)


In [9]:
import multiprocessing
import time
start = time.perf_counter()
processes = []
q = multiprocessing.Queue()
types = {'day':'d','month':'m','year':'y'}
# types = ['d']#,'m','y']
results=[]
def editDate(q,arg):
    print(arg)
    obj = np.array(train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%'+arg)), dtype=np.int8)
    q.put([arg,obj])
    # q.put(obj)
# def editDay(q):
#     day = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%d'))
#     q.put(day)
# def editMonth(q):
#     month = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%m'))
#     q.put(month)
# def editYear():
#     year = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%Y'))
#     return year

for i in types:
    # process=multiprocessing.Process(target=editDate, args=[q,i])
    process=multiprocessing.Process(target=editDate, args=[q,types[i]])
    print(i, process,"\n" )
    process.start() 
    processes.append(process)
for key in types.keys():
    # testFrame[key]=q.get()
    print(q.get())
print(len(processes))
for process in processes:
    process.join()

finish = time.perf_counter()

print(f'Finished in {round(finish-start, 2)} second(s)')

day <Process name='Process-1' parent=11869 initial> 

month <Process name='Process-2' parent=11869 initial> 

dyear <Process name='Process-3' parent=11869 initial> 

m

y
3
Finished in 46.05 second(s)


In [ ]:
train['month'] = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%m'))
train['year'] = train.date.apply(lambda x: datetime.strptime(x, '%d.%m.%Y').strftime('%Y'))
y = train['item_cnt_day']
train.drop(columns=['item_cnt_day', 'date'], axis=1, inplace=True)
# Obtain target and predictors
features = ['month', 'year', 'date_block_num','shop_id', 'item_price', 'item_category_id']